# Azure Machine Learning Pipeline with AutoMLStep 

This notebook is part of [Project 2: Operationalizing Machine Learning with Azure](https://github.com/dpbac/Operationalizing-Machine-Learning-with-Azure) of the `Udacity Nanodegree Program : Machine Learning Engineer with Azure`.

In this project you will continue working with the Bank Dataset introduced in [Project 1: Optimizing an ML Pipeline in Azure](https://github.com/dpbac/Optimizing-an-ML-Pipeline-in-Azure). We will use Azure in this project to configure a cloud-based machine learning product model, deploy it, and consume it.

**REVIEW AND COMPLETE ALL TEXT BELOW**

This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

A model is generated using AutoML for classifcation using the dataset available at https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv

## Introduction

In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook the following steps are performed

1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.



# Automated ML Experiment

In [ ]:
# Azure Machine Learning and Pipeline SDK-specific imports

import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

The config.json can be downloaded in the overview of Azure portal. (**Do I need to download?**)

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

In [ ]:
# it it is not working try like in the previous project

# from azureml.core import Workspace, Experiment

# # Initialize a workspace object for an existing Azure Machine Learning Workspace
# ws = Workspace.get("quick-starts-ws-127549")

## Create an Azure ML experiment

**REVIEW AND UPDATE TEXT**
**ATTENTION NAME OF THE EXPERIMENT**

Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created

In [ ]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing exp name
experiment_name = 'automl-experiment'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)

experiment

In [ ]:
# # Create a experiment
# exp = Experiment(workspace=ws, name="udacity-project2")

# run = exp.start_logging()

In [ ]:
dic_data = {'Workspace name': ws.name,
            'Azure region': ws.location,
            'Subscription id': ws.subscription_id,
            'Resource group': ws.resource_group,
            'Experiment Name': experiment.name}

df_data = pd.DataFrame.from_dict(data = dic_data, orient='index')

df_data.rename(columns={0:''}, inplace = True)
df_data

## Create or Attach an AmlCompute cluster

Now that we have initialized our workspace and created our experiment, it is time to define our resources. This means we need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run.

In this section you will create default compute clusters for use by the notebook and any other necessary operations we need. Here we get the default `AmlCompute` as your training compute resource.

In order to create a cluster we need to specify a compute configuration that defines the `type of machine` to be used and the `scalability behaviors`. Also, it is necessary to define the name of the cluster which must be unique within the workspace. This name is used to address the cluster later.

For this project we use a CPU cluster with following parameters:

* `type of the machine`:

    * `vm_size`: Defines the size of the virtual machine. We use here "STANDARD_DS12_V2" (more details [here](https://docs.microsoft.com/en-us/azure/cloud-services/cloud-services-sizes-specs#dv2-series))

* `Scalability behaviors`:

    * `min_nodes`: Sets minimun size of the cluster. Setting the minimum to 0 the cluster will shut down all nodes while not in use. If you use another value you are able to have faster start-up times, but you will also be billed when the cluster is not in use. In this experiment we define `min_nodes` as 1.

    * `max_nodes`: Sets the maximun size of the cluster. Larger number allows for more concurrency and a greater distributed processing of scale-out jobs.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Define CPU cluster name
cpu_cluster_name = "cpu-cluster"


# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cpu-cluster. Use it.")
except ComputeTargetException:
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_DS12_V2",
                                                           #vm_priority = 'lowpriority', # optional
                                                           min_nodes=1, # when innactive
                                                           max_nodes=4) # when busy

    # Create the cluster with the specified name and configuration
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
# # Wait for the cluster to complete, show the output log
# # cpu_cluster.wait_for_completion(show_output=True)
# cpu_cluster.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    cpu_cluster.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(cpu_cluster.get_status().serialize())

**CHECK IF THE FOLLOWING CELL IS STILL NECESSARY**

In [ ]:
# Check details about compute_targets (i.e. cpu_cluster)

compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

## Dataset

The code below try first to load the dataset from the workspace. The key is the dataset name and is used to find the dataset and load it.

If the key is not found, i.e., the dataset is not in the workspace the dataset is loaded from the web using the link given at the beginning of this notebook.

In [ ]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "BankMarketing"
description_text = "Bank Marketing DataSet for 2nd Project Udacity Azure ML Engineer."

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
        # Create TabularDataset using TabularDatasetFactory
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


#create a data frame
df = dataset.to_pandas_dataframe()

## Inspect Dataset

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.y.value_counts(normalize=True)

**INSERT COMMENTS ABOUT DATASET**

## Train model:

Now we use the [BankMarketing Dataset](https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv) to obtain a model by running AutoML.

AutoML is an automated machine learning capability included in Azure ML. It leverages the scalability of cloud compute to automatically try multiple pre-processing techniques and model-training algorithms in parallel to find the best performing supervised machine learning model for a dataset. In this particular case for the `BankMarketing` dataset, we have just loaded.

### Configure AutoML

In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 60, # define how long, in minutes, the experiment should continue to run.
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=cpu_cluster,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto', # as part of preprocessing, data guardrails and featurization steps are performed automatically.
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [ ]:
# automl_config = AutoMLConfig(
#     experiment_timeout_minutes=30,
#     task="classification",
#     primary_metric="accuracy",
#     training_data=df_train,
#     label_column_name='y',
#     n_cross_validations=5)

**USING MY FIRST PROJECT**

### Submitting Training Experiment

In [ ]:
automl_run = experiment.submit(config=automl_config, show_output=True)
automl_run

# TO CONTINUE WITH THE EXAMPLE NOTEBOOK USING PIPELINE